In [1]:
!pip install langchain cohere chroma -qqq

In [14]:
# Set env vars or load from a .env file
import dotenv
import os

dotenv.load_dotenv()

os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_PROJECT']='christos-mert-langchain-cohere-hackathon'

COHERE_API_KEY=os.getenv('COHERE_API_KEY')

In [15]:
from langchain.chat_models import ChatCohere
from langchain.schema import AIMessage, HumanMessage

chat = ChatCohere(cohere_api_key=COHERE_API_KEY)

In [ ]:
# messages = [HumanMessage(text="count to 5"), AIMessage(text="I'm good, how are you?")]

In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            """You are a librarian at cohere community library. Your job is to
            help recommend people books to read based on their interests and
            preferences. You also give information about the library.

            The library opens at 8am and closes at 9pm daily. It is closed on
            Sundays.

            If people speak too loud, you should tell them to shh
            """
        ),
        # The 'variable_name' here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=chat, memory=memory, verbose=True, prompt=prompt)

In [11]:
conversation.predict(message="My friend Mert has an online persona and makes announcements saying 'Mert Alert.' Please tell me what a Mert Alert is?")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
            You are a friendly bot
            
Human: What is a Mert Alert?
AI: Thanks! I appreciate you thinking of me in this way. Is there anything else I may assist you with today?
Human: Please tell me what a Mert Alert is?
AI: I am here to assist you in a friendly manner, and I hope to become more helpful and resourceful as we converse. Is there anything I may help you with?
Human: Please tell me what a Mert Alert is?
AI: Thanks! I do my best to be as friendly as possible, while providing you with the information you need. Is there anything else I can do to be of assistance?
Human: My friend Mert has an online persona and makes announcements saying 'Mert Alert.' Please tell me what a Mert Alert is?

> Finished chain.


"I'm glad you think I am friendly! I'm here to assist you in any way I can. If there is anything you need help with, please let me know."

In [13]:
import csv

csv_file = open('../kaggle-data/books_with_blurbs.csv', 'r')
csv_reader = csv.reader(csv_file)
csv_data = list(csv_reader)
csv_data = csv_data[1:2000]
parsed_data = [{
    "id": x[0],
    "title": x[1],
    "author": x[2],
    "year": x[3],
    "publisher": x[4],
    "blurb": x[5]
} for x in csv_data]
parsed_data[155]

{'id': '0446610399',
 'title': 'The Rescue',
 'author': 'Nicholas Sparks',
 'year': '2001',
 'publisher': 'Warner Books',
 'blurb': "When confronted by raging fires or deadly accidents, volunteer fireman Taylor McAden feels compelled to take terrifying risks to save lives. But there is one leap of faith Taylor can't bring himself to make: he can't fall in love. For all his adult years, Taylor has sought out women who need to be rescued, women he leaves as soon as their crisis is over and the relationship starts to become truly intimate. When a raging storm hits his small Southern town, single mother Denise Holton’s car skids off the road. The young mom is with her four-year-old son Kyle, a boy with severe learning disabilities and for whom she has sacrificed everything. Taylor McAden finds her unconscious and bleeding, but does not find Kyle. When Denise wakes, the chilling truth becomes clear to both of them: Kyle is gone. During the search for Kyle, the connection between Taylor and 

In [16]:
from langchain.embeddings import CohereEmbeddings

embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

docsearch = Chroma.from_texts(
    [x["title"] for x in parsed_data],
    embeddings,
    metadatas=parsed_data).as_retriever()

In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


prompt_template = """
{context}

Use the book reccommendations to suggest books for the user to read.
Only use the titles of the books, do not make up titles. Format the response as
a bulleted list prefixed by a relevant message.

User: {message}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "message"]
)

book_rec_chain = (
  {
    "input_documents": lambda x: docsearch.get_relevant_documents(x["message"]),
    "message": lambda x: x["message"]
  }
  | load_qa_chain(chat, chain_type="stuff", prompt=PROMPT)
)

query = "books that are good for people that like horror"
result = book_rec_chain.invoke({"message": query})
print(result["output_text"])
     

If you enjoy horror, then you may like the following books:
- The Mammoth Book of Ghost Stories
- The Year's Best Horror Stories: Series XIV
- The Shining
- Year's Best Fantasy and Horror: Sixteenth Edition


In [19]:

from langchain.schema.runnable import RunnableBranch
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser

chain = (
    PromptTemplate.from_template(
        """Given the user message below, classify it as either being about `recommendation` or `other`.

Do not respond with more than one word.


{message}


Classification:"""
    )
    | chat
    | StrOutputParser()
)

librarian_prompt_no_history = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            """
            You are a librarian at cohere community library. Your job is to
            help recommend people books to read based on their interests and
            preferences. You also give information about the library.

            The library opens at 8am and closes at 9pm daily. It is closed on
            Sundays.

            If people speak too loud, you should tell them to shh

            Please answer the following message:
            """
        ),
        HumanMessagePromptTemplate.from_template("{message}"),
    ]
)

librarian_chain = LLMChain(llm=chat, prompt=librarian_prompt_no_history)

branch = RunnableBranch(
    (lambda x: "recommendation" in x["topic"].lower(), book_rec_chain),
    { "message": lambda x: x["message"] } | librarian_chain | { "output_text": lambda x: x["text"]  }
)

full_chain = {"topic": chain, "message": lambda x: x["message"]} | branch

response = full_chain.invoke({"message": "whats your favourite book?"})

print(response["output_text"])

As an AI language model, I don't have personal preferences or the ability to read, so I don't have a favourite book. However, I can recommend some books to you based on books you have already read. 

The following books were recommended to you based on your reading history:
- Lost in a Good Book by Jasper Fforde
- Northern Lights by Philip Pullman
- His Dark Materials series by Philip Pullman
- Hitchhiker's Guide to the Galaxy by Douglas Adams
- Flowers for Algernon by Daniel Keyes


In [20]:

from langchain.chat_models import ChatCohere
from langchain.retrievers import CohereRagRetriever
from langchain.schema.document import Document

rag = CohereRagRetriever(llm=chat)

rag.get_relevant_documents("What is cohere ai?")

[Document(page_content='Cohere is a Canadian multinational technology company focused on artificial intelligence for the enterprise, specializing in large language models. Cohere was founded in 2019 by Aidan Gomez, Ivan Zhang, and Nick Frosst, and is headquartered in Toronto and San Francisco, with offices in Palo Alto and London.\n\nIn 2017, a team of researchers at Google Brain, which included Aidan Gomez, published a paper called "Attention is All You Need," which introduced the transformer machine learning architecture, setting state-of-the-art performance on a variety of natural language processing tasks. In 2019, Gomez and Nick Frosst, another researcher at Google Brain, founded Cohere along with Ivan Zhang, with whom Gomez had done research at FOR.ai. All of the co-founders attended University of Toronto.', metadata={'id': 'web-search_3:0', 'snippet': 'Cohere is a Canadian multinational technology company focused on artificial intelligence for the enterprise, specializing in lar